# Imports

In [1]:
import os
import numpy as np
import model_flux_ratio as mfr

from astropy.table import Table
from scipy.optimize import curve_fit

# Try scipy.optimize.curve_fit

## Read in our data

In [20]:
# Read in measured data (wavelength, flux ratios, and EWs)
#flux_ratios = Table.read(os.getcwd()+'/test_output_flux', format='ascii', delimiter=' ')
flux_ratios = Table.read(os.getcwd()+'/LeoP', format='ascii', delimiter=' ')

# Names of wavelenghts of interest for MCMC
y_names = ['HeI+H83890', 'HeI4027', 'Hd', 'Hg', 'HeI4472', 'Hb', 'HeI5017', 'HeI5877', 'Ha', 'HeI6679', 'HeI7067']

# Input parameters for fake spectra
input_vals = np.array([0.08, 18000, 2, 0.1, 1.0, 1.0, 1.0, 1e-2])

# 'Measured' data (generated flux ratios from input parameters)
EWs = np.array(flux_ratios['EW'])
EW_Hb = flux_ratios['EW'][np.where(flux_ratios['Wavelength'] == 4862.721)[0]]

y = np.array(flux_ratios['Flux Ratio'])
#y_error = np.array(flux_ratios['Flux Ratio'] * 0.02)
y_error = np.array(flux_ratios['Flux Ratio Errors'])
x = np.zeros(y.size)

## Emission lines of interest

In [21]:
# Balmer and Helium lines of interest for MCMC
balmer_lines = np.array([6564.612, 4862.721, 4341.684, 4102.891, 3890.166]) # Ha, Hb, Hg, Hd, H8
helium_lines = np.array([7067.198, 6679.994, 5877.299, 5017.079, 4472.755, 4027.328, 3890.151])

# Wavelengths we care about for MCMC
emis_lines = np.sort(np.concatenate((balmer_lines, helium_lines)))[1:] # [1:] to remove the duplicate ~3890 wavelength

## Range of parameters

In [22]:
# Range of values for 8 parameters: y_plus, temp, dens, c_Hb, a_H, a_He, tau_He, n_HI
min_y_plus, max_y_plus = 0.05, 0.1  # fraction of singly ionized He; y+ = He+/H+
min_temp, max_temp = 5000, 25000  # electron temperature (K)
min_log_dens, max_log_dens = 0, 3  # log10(electron density) (cm^-3)
min_c_Hb, max_c_Hb = 0, 0.5  # reddening
min_a_H, max_a_H = 0, 10  # underlying stellar H absorption (Angstroms)
min_a_He, max_a_He = 0, 4  # underlying stellar HeI absorption (Angstroms)
min_tau_He, max_tau_He = 0, 5  # optical depth; range of values from Izotov & Thuan 2010
min_log_xi, max_log_xi = -6, -0.969 # ratio of neutral to singly ionized hydrogen density
#min_n_HI, max_n_HI = 1e-4, 1e-1  # neutral hydrogen density (cm^-3)

## Model

In [23]:
def get_model(x, y_plus, temp, log_dens, c_Hb, a_H, a_He, tau_He, log_xi):

    model_flux = np.zeros(len(x))
    dens = 10 ** log_dens
    xi = 10 ** log_xi
    #xi = n_HI / dens

    # Some values, calculated at Hbeta, for later use
    collisional_to_recomb_Hbeta = mfr.hydrogen_collision_to_recomb(xi, balmer_lines[1], temp)
    f_lambda_at_Hbeta = mfr.f_lambda_avg_interp(balmer_lines[1])

    for w in range(len(emis_lines)):
        # Determine if working with hydrogen or helium line; within 3 Angstroms is arbitrary but should cover difference in vacuum vs air wavelength
        nearest_wave = emis_lines[np.where(np.abs(emis_lines - emis_lines[w]) < 3)[0]][0]
        # The above line is redundant, but allows for cases where emis_lines[w] is some other array, say waves_of_interest[w], 
        # and not exactly at the wavelengths given in the emis_lines array (which is concatenated from arrays balmer_lines and helium_lines)

        # Any Balmer line besides the blended HeI+H8 line (H8 at 3890.166)
        if nearest_wave in balmer_lines and nearest_wave != 3890.166:
            line_species = 'hydrogen'
            
            emissivity_ratio = mfr.hydrogen_emissivity(emis_lines[w], temp, dens)
            a_H_at_wave = mfr.stellar_absorption(emis_lines[w], a_H, ion=line_species)            
            collisional_to_recomb_ratio = mfr.hydrogen_collision_to_recomb(xi, emis_lines[w], temp)            
            reddening_function = ( mfr.f_lambda_avg_interp(emis_lines[w]) / f_lambda_at_Hbeta ) - 1.            

            flux = emissivity_ratio * ( ( (EW_Hb + a_H)/(EW_Hb) ) / ( (EWs[w] + a_H_at_wave)/(EWs[w]) ) ) * \
                    ( (1 + collisional_to_recomb_ratio) / (1 + collisional_to_recomb_Hbeta) ) * \
                    10**-(reddening_function * c_Hb)
                    
        # Any HeI line besides the blended HeI+H8 line (HeI at 3890.151)
        elif nearest_wave in helium_lines and nearest_wave != 3890.151:
            line_species = 'helium'
            
            emissivity_ratio = mfr.helium_emissivity(emis_lines[w], temp, dens)            
            a_He_at_wave = mfr.stellar_absorption(emis_lines[w], a_He, ion=line_species)            
            optical_depth_at_wave = mfr.optical_depth_function(emis_lines[w], temp, dens, tau_He)            
            collisional_to_recomb_ratio = mfr.helium_collision_to_recomb(emis_lines[w], temp, dens)            
            reddening_function = ( mfr.f_lambda_avg_interp(emis_lines[w]) / f_lambda_at_Hbeta ) - 1.

            flux = y_plus * emissivity_ratio * ( ( (EW_Hb + a_H)/(EW_Hb) ) / ( (EWs[w] + a_He_at_wave)/(EWs[w]) ) ) * \
                    optical_depth_at_wave * ( (1 + collisional_to_recomb_ratio) / (1 + collisional_to_recomb_Hbeta) ) * \
                    10**-(reddening_function * c_Hb)
        
        # The blended HeI+H8 line
        elif nearest_wave == 3890.151 or nearest_wave == 3890.166:
            # HeI 3890.151 contribution:
            line_species = 'helium'
            
            emissivity_ratio = mfr.helium_emissivity(emis_lines[w], temp, dens)
            a_He_at_wave = mfr.stellar_absorption(emis_lines[w], a_He, ion=line_species)            
            optical_depth_at_wave = mfr.optical_depth_function(emis_lines[w], temp, dens, tau_He)            
            collisional_to_recomb_ratio = mfr.helium_collision_to_recomb(emis_lines[w], temp, dens)            
            reddening_function = ( mfr.f_lambda_avg_interp(emis_lines[w]) / f_lambda_at_Hbeta ) - 1.

            flux = y_plus * emissivity_ratio * ( ( (EW_Hb + a_H)/(EW_Hb) ) / ( (EWs[w] + a_He_at_wave)/(EWs[w]) ) ) * \
                    optical_depth_at_wave * ( (1 + collisional_to_recomb_ratio) / (1 + collisional_to_recomb_Hbeta) ) * \
                    10**-(reddening_function * c_Hb)
                    
            # H8 contribution:
            line_species = 'hydrogen'

            emissivity_ratio = mfr.hydrogen_emissivity(emis_lines[w], temp, dens)
            a_H_at_wave = mfr.stellar_absorption(emis_lines[w], a_H, ion=line_species)            
            collisional_to_recomb_factor = np.exp(( -13.6 * ((1/5**2)-(1/8**2)) ) / (8.6173303e-5 * temp)) # scale factor for going from C/R(Hg) to C/R(H8)
            collisional_to_recomb_ratio = collisional_to_recomb_factor * mfr.hydrogen_collision_to_recomb(xi, 4341.684, temp) # Calculate C/R(Hg) and multiply by above scale factor
            reddening_function = ( mfr.f_lambda_avg_interp(emis_lines[w]) / f_lambda_at_Hbeta ) - 1.            

            flux += emissivity_ratio * ( ( (EW_Hb + a_H)/(EW_Hb) ) / ( (EWs[w] + a_H_at_wave)/(EWs[w]) ) ) * \
                    ( (1 + collisional_to_recomb_ratio) / (1 + collisional_to_recomb_Hbeta) ) * \
                    10**-(reddening_function * c_Hb)

        model_flux[w] = flux

    return model_flux

## Initial guess

In [28]:
pos = [np.random.uniform(min_y_plus, max_y_plus), np.random.uniform(min_temp, max_temp), \
                np.random.uniform(min_log_dens, max_log_dens), np.random.uniform(min_c_Hb, max_c_Hb), \
                np.random.uniform(min_a_H, max_a_H), np.random.uniform(min_a_He, max_a_He), \
                np.random.uniform(min_tau_He, max_tau_He), np.random.uniform(min_log_xi, max_log_xi)]

print ('Initial guess:', pos)

Initial guess: [0.05120661096825288, 6812.780016700919, 0.11749559932599118, 0.44705425528287007, 8.199054459031519, 3.4256279902779, 1.1733335082243785, -3.467519431952725]


In [29]:
test_model = get_model(x, pos[0], pos[1], pos[2], pos[3], pos[4], pos[5], pos[6], pos[7])

print ('Flux ratios from initial positions:', test_model)

Flux ratios from initial positions: [9.36097175e-02 1.89586642e-03 1.75003296e-01 3.70499772e-01
 1.42638802e-02 1.00000000e+00 9.08794123e-03 9.11298293e-02
 4.28935349e+00 2.87258251e-02 1.79053224e-02]


## curve_fit

In [35]:
best_params, covar = curve_fit(get_model, x, y, p0=pos, sigma=y_error, \
                        bounds=((min_y_plus, min_temp, min_log_dens, min_c_Hb, min_a_H, min_a_He, min_tau_He, min_log_xi), \
                        (max_y_plus, max_temp, max_log_dens, max_c_Hb, max_a_H, max_a_He, max_tau_He, max_log_xi)))

In [36]:
best_params

array([ 7.86460406e-02,  1.43074730e+04,  3.54539939e-01,  6.65236987e-02,
        2.40383108e+00,  4.03378600e-01,  9.36280761e-01, -5.83369919e+00])

# Flux ratios from best fit parameters

In [37]:
best_fit_model = get_model(x, best_params[0], best_params[1], best_params[2], best_params[3], \
                       best_params[4], best_params[5], best_params[6], best_params[7])



In [38]:
best_fit_model - flux_ratios['Flux Ratio']

0.0008515134941122215
0.0005144732384681772
-0.0068605535158415
0.018993374200058588
0.0012968114588568294
0.0
-0.003982112914470915
0.00010381254565729192
0.024698503638293623
-0.0004984489168539455
0.0002035779001764021


In [39]:
best_fit_model

array([0.17585151, 0.01151447, 0.23913945, 0.44999337, 0.03429681,
       1.        , 0.02201789, 0.10310381, 2.9736985 , 0.02950155,
       0.02520358])